## Check autograd forces against analytical forces
This notebook compares the forces computed with automatic differentiation (with respect to bond vectors, not atom coordinates) against the analytical forces for the [Al99 Embedded Atom potential](https://www.ctcms.nist.gov/potentials/entry/1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al/), as implemented by [ASE](https://wiki.fysik.dtu.dk/ase/ase/atoms.html).

First, fetch the EAM potential data from interatomic potentials repository:

In [1]:
!curl https://www.ctcms.nist.gov/potentials/Download/1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al/2/Al99.eam.alloy -o Al99.eam.alloy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  762k  100  762k    0     0   467k      0  0:00:01  0:00:01 --:--:--  467k


Gradient checking is best done in 64-bit floating point precision:

In [7]:
from pathlib import Path

import torch

import ase.io
from ase.build import bulk
from ase.calculators.eam import EAM
from ase.calculators.lammpsrun import LAMMPS

import nfflr
from nfflr.data import graph
from nfflr.models.classical.eam import TorchEAM

torch.set_default_dtype(torch.float64)

We set up a small FCC aluminum system and add a small amount of jitter to the atomic coordinates:

In [2]:
# set up a small bulk Aluminum calculation
a = 4.05  
atoms = bulk("Al", "fcc", a) * [4, 4, 4]
atoms.rattle(stdev=0.05)
atoms.wrap()
ase_eam = EAM(potential="Al99.eam.alloy")
atoms.set_calculator(ase_eam)

# set up pytorch version
# al = nfflr.Atoms(al_ase.get_cell().array, al_ase.get_positions(), al_ase.numbers)


/var/folders/p4/50p5qpjx0snc7sqp9sp7qd1r001tvs/T/ipykernel_66668/2909281923.py:7: DeprecationWarning: Please use atoms.calc = calc
  atoms.set_calculator(ase_eam)


The ASE implementation computes the forces using the analytical gradients of the EAM spline components, while the pytorch implementation uses automatic differentiation to compute the gradient of the energy with respect to the bond displacement vectors. A [sum reduction ](https://github.com/usnistgov/nfflr/blob/2026152caa6beab2fa0dcf066288223726e78215/nfflr/models/utils.py#L43-L51) is used to aggregate these into the force components on individual atoms.

Both the energies and the force components on all the atoms match to within floating point precision:

In [6]:
torch_eam = TorchEAM("Al99.eam.alloy", dtype=torch.float64)

# construct radius graph matching ASE cutoff
g = graph.periodic_radius_graph(nfflr.Atoms(atoms), r=torch_eam.data.cutoff, dtype=torch.float64)

# evaluate energies and forces with pytorch implementation
out_nff = torch_eam(g)
e_dgl, force_dgl = out_nff["energy"].detach(), out_nff["forces"].detach()

# evaluate energy and forces with ASE
e_ase = atoms.get_potential_energy()
force_ase = atoms.get_forces()

print(f"{atoms.get_potential_energy()=}")
print(f"{e_dgl.detach().item()=}")
print(f"potential energy matches: {np.isclose(e_ase, e_dgl.item())}")
print(f"force components match? : {np.isclose(force_ase, force_dgl).all()}")

atoms.get_potential_energy()=-214.15885773313198
e_dgl.detach().item()=-214.15885773313158
potential energy matches: True
force components match? : True


/Users/bld/.pyenv/versions/3.10.9/envs/nfflr/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


## Tersoff potential
This section performs the same diagnostic using the Tersoff potential, as implemented by LAMMPS.
This demonstrates that forces computed by autograd with respect to relative position vectors reduce to the correct atomic forces.

The parameters used correspond to https://www.ctcms.nist.gov/potentials/entry/1988--Tersoff-J--Si-b/:

In [9]:
!curl https://www.ctcms.nist.gov/potentials/Download/1988--Tersoff-J--Si-b/1/1988_Si\(B\).tersoff -o Si.tersoff
!cat Si.tersoff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   532  100   532    0     0     49      0  0:00:10  0:00:10 --:--:--   143 0 --:--:--  0:00:02 --:--:--     0     0 --:--:--  0:00:03 --:--:--     0
# Parameters for the Tersoff Si(B) potential. CITATION: J. Tersoff, Phys. Rev. B 37, 6991 (1988)

# Parameter values verified by Lucas Hale.
# Identical values in Si.tersoff of August 22, 2018 LAMMPS distribution.
# Identical values in openKIM model MO_245095684871_001 parameter file.

# Values are in LAMMPS "metal" units.

# e1 e2 e3 m   gamma lambda3 c      d      costheta0 n      beta    lambda2 B      R   D   lambda1 A
  Si Si Si 3.0 1.0   1.3258  4.8381 2.0417 0.0       22.956 0.33675 1.3258  95.373 3.0 0.2 3.2394  3264.7

### Small silicon system
Now, set up a small Si system and a LAMMPS calculator to serve as the reference Tersoff potential implementation:

In [8]:
# set up a small Si example crystal
si_ase = bulk("Si", "diamond", 5.43) * [4, 4, 4]
si_ase.rattle(stdev=0.01, seed=36)
si_ase.wrap()

# configure LAMMPS Tersoff potential
calc = LAMMPS()
calc.parameters["command"] = "/opt/homebrew/bin/lmp_serial"
calc.parameters["files"] = ["Si.tersoff"]
calc.parameters["binary_dump"] = False
calc.parameters.update(
    {"pair_style": "tersoff", "pair_coeff": ["* * Si.tersoff Si"]}
    )
si_ase.set_calculator(calc)

# calculate Tersoff energy and forces with LAMMPS
e_lammps = si_ase.get_potential_energy()
f_lammps = si_ase.get_forces()

print(f"{e_lammps=}")
print(f"{f_lammps[:5]=}")

e_lammps=-592.3985671491682
f_lammps[:5]=array([[-0.20522987, -0.26851818,  0.14179856],
       [ 0.06219078,  0.05221548,  0.0728464 ],
       [ 0.03842853, -0.02849059,  0.04544221],
       [-0.21198316,  0.12308603, -0.00683117],
       [ 0.5269759 , -0.12608296,  0.29867076]])


/var/folders/p4/50p5qpjx0snc7sqp9sp7qd1r001tvs/T/ipykernel_66668/4189360319.py:14: DeprecationWarning: Please use atoms.calc = calc
  si_ase.set_calculator(calc)


And 

In [11]:
g = graph.periodic_radius_graph(nfflr.Atoms(si_ase), r=3.5, dtype=torch.float64)
tersoff = nfflr.models.Tersoff()
out = tersoff(g)
e_tersoff = out["energy"].detach()
f_tersoff = out["forces"].detach()
stress_tersoff = -out["virial"].detach() / si_ase.get_volume()

print(f"{e_tersoff=}")
print(f"{f_tersoff[:5]=}")
print(f"{stress_tersoff=}")


e_tersoff=tensor(-592.3986)
f_tersoff[:5]=tensor([[-0.2052, -0.2685,  0.1418],
        [ 0.0622,  0.0522,  0.0728],
        [ 0.0384, -0.0285,  0.0454],
        [-0.2120,  0.1231, -0.0068],
        [ 0.5270, -0.1261,  0.2987]])
stress_tersoff=tensor([[[-6.5235e-04,  1.6092e-03,  4.2400e-04],
         [ 1.6092e-03, -6.5744e-04,  2.0972e-06],
         [ 4.2400e-04,  2.0972e-06, -6.4832e-04]]])


The total energies match to within numerical precision:

In [12]:
np.isclose(e_tersoff, e_lammps)

True

And so do the forces (using the numerical precision settings used by [torch.autograd.gradcheck](https://pytorch.org/docs/stable/generated/torch.autograd.gradcheck.html):

In [13]:
np.isclose(f_lammps, f_tersoff, atol=1e-05, rtol=0.001).all()

True

The largest force component discrepancy is about $7 \times 10^{-7} \; eV/\mathrm{\AA}$

In [14]:
(f_lammps - f_tersoff.numpy()).max()

6.749736075409296e-07

And the stress tensor values also match:

In [15]:
si_ase.get_stress()

array([-6.52352915e-04, -6.57444097e-04, -6.48324331e-04,  2.09715783e-06,
        4.23995521e-04,  1.60922348e-03])

In [16]:
np.isclose(
    stress_tersoff, 
    ase.stress.voigt_6_to_full_3x3_stress(si_ase.get_stress())
)

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]])